In [1]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook
import time
from datetime import datetime, date
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import re
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import sys
# import gmplot package 
import gmplot 
import random
from openpyxl import load_workbook


In [2]:
land1 = pd.read_excel(r"C:\Users\Lenovo\數據分析\天下專案\land.xlsx")
area_df_clean = pd.read_excel(r"C:\Users\Lenovo\數據分析\天下專案\area_df_clean.xlsx")
land = pd.merge(land1, area_df_clean, left_on="sect", right_on="區名").drop('區名',axis=1)
land1

,city,town,sect,number
0,苗栗縣,通霄鎮,大坪頂段,140-18
1,苗栗縣,通霄鎮,大坪頂段,140-30
2,苗栗縣,通霄鎮,大坪頂段,323
3,苗栗縣,苑裡鎮,南勢林坑段,1061
4,苗栗縣,苑裡鎮,南勢林坑段,1068
5,苗栗縣,苑裡鎮,南勢林坑段,494
6,苗栗縣,通霄鎮,北勢窩段,538
7,苗栗縣,通霄鎮,北勢窩段,539
8,苗栗縣,通霄鎮,北勢窩段,539-1
9,苗栗縣,通霄鎮,北勢窩段,540


In [93]:
browser = webdriver.Chrome(executable_path = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe")

info = pd.DataFrame(columns = ['city','town','sect','number','lalo'])

for i in tqdm_notebook(range(1,3)):
    browser.get("https://easymap.land.moi.gov.tw/R02/Index")
    time.sleep(2)
    #browser.find_element_by_id('').click()  # 实例化Select
    try:
        browser.find_element_by_id('notShowSysinfo_id').click()
    except:
        pass
    Select(browser.find_element_by_id('select_city_id')).select_by_visible_text(land['city'][i])
    time.sleep(4)
    
    browser.find_element_by_id('select_town_id').click()
    #點開下拉式選單
    Select(browser.find_element_by_id('select_town_id')).select_by_visible_text(land['town'][i])
    #點擊鄉鎮的名字
    time.sleep(4)
    
    browser.find_element_by_id('select_sect_id').click()
    #點開區段下拉式選單
    s1 = Select(browser.find_element_by_id('select_sect_id'))
    #設置選取id範圍
    for each_option in s1.options:
        if land['sect'][i] in each_option.text:
            s1.select_by_visible_text(each_option.text)
    #每一個在id範圍中的值去跑，如果有跟區段名字有部分吻合的就用那個text去點擊
    time.sleep(5)
    
    browser.find_element_by_id('landno').send_keys(land['number'][i])
    browser.find_element_by_id('land_button').click()
    actionChains = ActionChains(browser)
    time.sleep(2)
    source = browser.find_element_by_id('OpenLayers.Layer.Vector_28_vroot')
    time.sleep(10)
    actionChains.context_click(source).perform()
    browser.find_element_by_xpath('/html/body/ul/li[2]').click()
    browser.find_element_by_id('OpenLayers.Layer.Vector_28_vroot').click()
    soup7 = BeautifulSoup(browser.page_source, "html.parser") 
    blocks = soup7.find("span", {"id": "coordDisplayLonLat"})
    pngname = str(land['town'][i]) + str(land['sect'][i]) + str(land['number'][i])
    filename = "C:/Users/Lenovo/數據分析/天下專案/" + pngname + ".png"
    browser.save_screenshot(filename)
    com_info2 = pd.DataFrame({"city":[land['city'][i]], 
                    "town":[land['town'][i]],
                   "sect":[land['sect'][i]],
                   "number":land['number'][i],          
                   'lalo':[blocks.text]}) 
    info = pd.concat([info, com_info2], axis = 0)
    #用一開始建立的info來跟後面的com_info2使用 concat 合併 axis=0 為直向合併
    info = info.reset_index(drop = True)

C:\Users\Lenovo\anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [123]:
info[['latitude','longitude']] = info.lalo.str.split(",",expand=True,)
info['latitude'] = pd.to_numeric(info['latitude'])
info['longitude'] = pd.to_numeric(info['longitude'])
info

,city,town,sect,number,lalo,latitude,longitude
0,苗栗縣,通霄鎮,大坪頂段,140-30,"24.44724,120.70499",24.44724,120.70499
1,苗栗縣,通霄鎮,大坪頂段,323,"24.44328,120.70565",24.44328,120.70565


In [121]:
for index, row in info.iterrows():
    print(row['city'], row['lalo'])

苗栗縣 24.44724,120.70499
苗栗縣 24.44328,120.70565


In [127]:
type(info['longitude'])

pandas.core.series.Series

In [146]:
#Open the file to be written.
f = open('測試.kml','w',encoding = 'UTF-8')

#Writing the kml file.
f.write("<?xml version='1.0' encoding='UTF-8'?>\n")
f.write("<kml xmlns='http://earth.google.com/kml/2.1'>\n")
f.write("<Document>\n")
for index, row in info.iterrows():
    f.write("   <Placemark>\n")
    f.write("       <name>" + str(row['city'])+str(row['town'])+str(row['sect'])+str(row['number']) + "</name>\n")
    f.write("       <description>" + str(row['city'])+str(row['town'])+str(row['sect'])+str(row['number'])+ "</description>\n")
    f.write("       <Point>\n")
    f.write("           <coordinates>" + str(row['longitude']) + "," + str(row['latitude']) + ","  + "</coordinates>\n")
    #因為,是文字，所以不能直接用數字加上去，要轉換成string
    f.write("       </Point>\n")
    f.write("   </Placemark>\n")
f.write("</Document>\n")
f.write("</kml>\n")
f.close()

In [52]:
#爬取區域資料
browser = webdriver.Chrome(executable_path = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe")

area_df = pd.DataFrame(columns = ['城市','區域','區號名'])

for i in tqdm(range(1,3)):
    for y in tqdm(range(1,3)):
        browser.get("https://easymap.land.moi.gov.tw/R02/Index")
        time.sleep(2)
        try:
            browser.find_element_by_id('notShowSysinfo_id').click()
        except:
            pass
        try:
            s1 = Select(browser.find_element_by_id('select_city_id'))
            s1.select_by_index(i)
            browser.find_element_by_id('select_town_id').click()
            s2 = Select(browser.find_element_by_id('select_town_id'))
            s2.select_by_index(y)
            time.sleep(4)
            soup7 = BeautifulSoup(browser.page_source, "html.parser")
            city = s1.first_selected_option.text
            town = s2.first_selected_option.text
            blocks = soup7.find("select", {"id": "select_sect_id"})
            area_info = blocks.text.split("(")
            #根據(符號來分割成list
            area = pd.DataFrame (area_info,columns=['區號名'])
            area = area.drop(area.index[0])
            #轉換成dataframe
            area_df = pd.concat([area_df,area], axis = 0)
            #用一開始建立的info來跟後面的com_info2使用 concat 合併 axis=0 為直向合併
            area_df  = area_df.reset_index(drop = True)
        except:
            pass
         # 隨機暫停 1~5 秒
        time.sleep(random.uniform(1, 5))
        

In [70]:
#area_df['區號名'] = '('+ area_df['區號名'].astype(str)
#area_df['區號名'] = area_df['區號名'].str.replace("(","")
#area_df['區號名'] = '('+ area_df['區號名'].astype(str)
area_df_clean = pd.DataFrame(area_df['區號名'].str.split(')',1).tolist(),
                                 columns = ['區號','區名'])
#area_df_clean['區號'] =  area_df_clean['區號'].astype(str)+')'
#area_df_clean.to_excel("area_df_clean.xlsx")

In [30]:
inputfile = info[['number','latitude','longitude']]

In [85]:
#把dataframe裡面的某欄變成list
latitude_list = info['latitude'].tolist()
longitude_list = info['longitude'].tolist()


TypeError: cannot convert the series to <class 'float'>

In [91]:
#把dataframe裡面的某欄變成list
latitude_list = info['latitude'].tolist()
longitude_list = info['longitude'].tolist()
  
gmap5 = gmplot.GoogleMapPlotter(24.44724, 
                                120.70499, 13) 

gmap5.heatmap(latitude_list, longitude_list) 
  
gmap5.draw( r"C:\Users\Lenovo\數據分析\天下專案\map15.html" ) 

In [32]:
inputfile.to_csv("geocoded-placenames.csv")

In [1]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [32]:
import anvil.server
anvil.server.connect("2SQQ6GADNM655OXSCZ4TFL3K-OPQOLYC6XIRDMJRX")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [33]:
import pandas as pd
import anvil.media

@anvil.server.callable
def store_data(file):
  with anvil.media.TempFile(file) as file_name:
    if file.content_type == 'text/csv':
      df = pd.read_csv(file_name)
    else:
      df = pd.read_excel(file_name)
    for d in df.to_dict(orient="records"):
      # d is now a dict of {columnname -> value} for this row
      # We use Python's **kwargs syntax to pass the whole dict as
      # keyword arguments
      app_tables.your_table_name_here.add_row(**d)